**7.- Diseña un modelo en Keras que utilice capas densas, LSTM, GRU, y convolucionales para predecir los valores de cierre Apple utilizando como entrada los valores de apertura de los datos Amazon, Microsoft, Meta (facebook), Apple y Google de los 4 días anteriores. Compáralo con un modelo con una capa lineal.**


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Layer, Dense, GRU, Conv1D, Flatten, Dropout, BatchNormalization, MaxPooling2D, LSTM
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [7]:
#Leemos los datos de apple
apple = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\aa.us.txt", delimiter=",")
amazon = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\amzn.us.txt", delimiter=",")
microsoft = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\msft.us.txt", delimiter=",")
facebook = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\fb.us.txt", delimiter=",")
google = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\googl.us.txt", delimiter=",")
#Me quedo con los datos de apertura


In [8]:
#Convertimos la columna Date como indice
for asset in [apple, amazon, microsoft, google, facebook]:
    asset.index = pd.to_datetime(asset.Date)

In [9]:
#Filtramos por el indice para homogeneizar los datos
data_close = apple[facebook.index[0]:].Close
apple = apple[facebook.index[0]:].Open
amazon = amazon[facebook.index[0]:].Open
microsoft = microsoft[facebook.index[0]:].Open
google = google[facebook.index[0]:].Open
facebook = facebook.Open


In [10]:
#Me genero una matriz de ceros con las dimensiones que quiero, n_ejemplos x 5 datos. 
LAG = 5
data = np.zeros((apple.shape[0]- LAG, LAG, 5))
close = np.zeros((data_close.shape[0]- LAG, LAG, 5))

In [11]:
#Unimos los datos en un único dataframe
for i in range(0,(apple.shape[0] - LAG)):
    data[i,:,0] = apple.iloc[i:i+LAG]
    data[i,:,1] = amazon.iloc[i:i+LAG]
    data[i,:,2] = microsoft.iloc[i:i+LAG]
    data[i,:,3] = google.iloc[i:i+LAG]
    data[i,:,4] = facebook.iloc[i:i+LAG]

In [12]:
data.shape

(1376, 5, 5)

In [13]:
x_train = data[:round(data.shape[0]*0.7),:-1,:]
x_test = data[round(data.shape[0]*0.7):,:-1,:]
y_train = close[:round(data.shape[0]*0.7),-1,0]
y_test = close[round(data.shape[0]*0.7):,-1,0]

In [14]:
model = Sequential()
model.add(LSTM(50, activation = 'relu', input_shape = (4,5), return_sequences = True))
model.add(GRU(40, activation = 'relu',  return_sequences = True))
model.add(Conv1D(30, kernel_size = 3, activation = 'relu',  
                 kernel_regularizer=regularizers.l2(0.01),padding='same'))
model.add(BatchNormalization())
model.add(LSTM(50, activation = 'relu', return_sequences = True))
model.add(GRU(20, activation = 'relu', return_sequences = True))
model.add(Conv1D(10, kernel_size = 3, activation = 'relu', 
                 kernel_regularizer=regularizers.l2(0.01),padding='same'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(1, activation = 'linear'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 50)             11200     
                                                                 
 gru (GRU)                   (None, 4, 40)             11040     
                                                                 
 conv1d (Conv1D)             (None, 4, 30)             3630      
                                                                 
 batch_normalization (BatchN  (None, 4, 30)            120       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 4, 50)             16200     
                                                                 
 gru_1 (GRU)                 (None, 4, 20)             4320      
                                                        

In [16]:
model.compile(optimizer='adam',loss='mae')

In [18]:
history_convulcional = model.fit(x_train, y_train,  epochs=50)

Epoch 1/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0509
Epoch 2/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0550
Epoch 3/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0560
Epoch 4/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0515
Epoch 5/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0481
Epoch 6/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0465
Epoch 7/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0508
Epoch 8/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0505
Epoch 9/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0586
Epoch 10/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0430
Epoch 11/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0445
Epoch 12/50
31/31 [==============================] - 0s 5ms/step - loss: 0.0428
Epoch 13/50
31/31 [==============================

In [19]:
model.evaluate(x_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.1908


0.19078843295574188

In [27]:
x_test.shape

(413, 4, 5)

Para compararlo con un modelo lineal es necesario aplanar los datos de entrada, puesto que tendrán una dimensión de n,4,5

In [39]:
model = Sequential()

model.add(Flatten(input_shape=(4,5)))
model.add(Dense(5))
model.add(Dense(1, activation = 'linear'))


In [40]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 20)                0         
                                                                 
 dense_6 (Dense)             (None, 5)                 105       
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(optimizer='adam',loss='mae')

In [42]:
history_convulcional = model.fit(x_train, y_train,  epochs=50)

Epoch 1/50
31/31 [==============================] - 0s 667us/step - loss: 221.0817
Epoch 2/50
31/31 [==============================] - 0s 600us/step - loss: 33.2552
Epoch 3/50
31/31 [==============================] - 0s 600us/step - loss: 26.1608
Epoch 4/50
31/31 [==============================] - 0s 600us/step - loss: 23.8328
Epoch 5/50
31/31 [==============================] - 0s 600us/step - loss: 21.5821
Epoch 6/50
31/31 [==============================] - 0s 600us/step - loss: 19.3641
Epoch 7/50
31/31 [==============================] - 0s 600us/step - loss: 17.0421
Epoch 8/50
31/31 [==============================] - 0s 600us/step - loss: 14.5433
Epoch 9/50
31/31 [==============================] - 0s 600us/step - loss: 12.3126
Epoch 10/50
31/31 [==============================] - 0s 633us/step - loss: 9.8589
Epoch 11/50
31/31 [==============================] - 0s 633us/step - loss: 8.0272
Epoch 12/50
31/31 [==============================] - 0s 600us/step - loss: 6.7949
Epoch 13/50
31/

In [43]:
model.evaluate(x_test, y_test)

13/13 [==============================] - 0s 750us/step - loss: 4.5155


4.515481472015381

Vemos que con las capas densas obtenemos un resultado mucho peor que con las capas LSTM, GRU, y convulcionales